# HW 1
The homework assignments is on calibrating the Inertial sensor in our cell phones via the 6-position method. The first part will be to use recorded data to calibrate the accelerometer, the second part will be to calibrate the gyroscopes and the third part will be to analyze recorded data of "shaking the smartphone". Finally, we will compare different phones with one another and with our expectations from their specifications. 


In [ ]:
import sys
import os
import numpy as np
import pandas as pd

sys.path.append('../src')
from ins import Accelerometer
from ins import Gyroscope
sys.path.append('../notebooks')
np.set_printoptions(precision=4, suppress=True)

path_accel_300sec_face_up = os.path.join("..", "data", "Acceleration_300sec_FaceUp")
path_accel_300sec_face_down = os.path.join("..", "data", "Acceleration_300sec_FaceDown")
path_accel_300sec_side_positive = os.path.join("..", "data", "Acceleration_300sec_SidePositive")
path_accel_300sec_side_negative = os.path.join("..", "data", "Acceleration_300sec_SideNegative")
path_accel_300sec_vertical_front = os.path.join("..", "data", "Acceleration_300sec_VerticalFront")
path_accel_300sec_vertical_back = os.path.join("..", "data", "Acceleration_300sec_VerticalBack")

accel_300sec_face_up = pd.read_csv(os.path.join(path_accel_300sec_face_up, "Raw Data.csv")) # This had a delimiter of ","
accel_300sec_face_down = pd.read_csv(os.path.join(path_accel_300sec_face_down, "Raw Data.csv"), sep="\t", engine="python")
accel_300sec_side_positive = pd.read_csv(os.path.join(path_accel_300sec_side_positive, "Raw Data.csv"), sep="\t", engine="python")
accel_300sec_side_negative = pd.read_csv(os.path.join(path_accel_300sec_side_negative, "Raw Data.csv"), sep="\t", engine="python")
accel_300sec_vertical_front = pd.read_csv(os.path.join(path_accel_300sec_vertical_front, "Raw Data.csv"), sep="\t", engine="python")
accel_300sec_vertical_back = pd.read_csv(os.path.join(path_accel_300sec_vertical_back, "Raw Data.csv"), sep="\t", engine="python")

In [2]:
# get mean specific forces for each axis
fz_up = accel_300sec_face_up[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fz_down = accel_300sec_face_down[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_up = accel_300sec_side_positive[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_down = accel_300sec_side_negative[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_up = accel_300sec_vertical_front[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_down = accel_300sec_vertical_back[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])


### Part 1 - Calculate bias and scale factor assuming no misalignment

In [3]:
g_true = 9.81 # m/s^2
a = Accelerometer(name="Elad's cell phone")
a.set_calibration_data(
    fx_up=fx_up, fx_down=fx_down, 
    fy_up=fy_up, fy_down=fy_down, 
    fz_up=fz_up, fz_down=fz_down, 
    g=g_true)


In [4]:
bias = a.get_bias()
scale_factor = a.get_scale_factor()

In [5]:
print(f"Stats for {a.get_sensor_name()} sensors:")
print(f"bias x: {bias[0][0]}\nbias y: {bias[1][0]}\nbias z: {bias[2][0]}")
print(f"scale factor x: {scale_factor[0,0]}\nscale factor y: {scale_factor[1,1]}\nscale factor z: {scale_factor[2,2]}")

Stats for Elad's cell phone sensors:
bias x: -0.115777164232437
bias y: -0.10505097597297208
bias z: -0.05714166333679582
scale factor x: -0.0023240669520969504
scale factor y: -0.0005669978446983048
scale factor z: 0.009417891979172198


### Part 2 - Calculate Scale Factor, Bias and Misalignment using LSM

In [6]:
# Calculate the errors matrix M = [SF + M | b]
M = a.get_M_errors_matrix()

In [7]:
print(f"Stats for {a.get_sensor_name()} sensors:")
print(f"M Matrix:\n{M}")

Stats for Elad's cell phone sensors:
M Matrix:
[[-0.0023 -0.0069 -0.0047 -0.1132]
 [ 0.0028 -0.0006  0.0055 -0.0857]
 [ 0.0139  0.0002  0.0094 -0.091 ]]
